In [6]:
!pip install opencv-python


   ---------------------------------------- 0.0/38.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/38.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/38.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/38.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/38.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/38.8 MB ? eta -:--:--
   ---------------------------------------- 0.1/38.8 MB 350.1 kB/s eta 0:01:51
   ---------------------------------------- 0.1/38.8 MB 350.1 kB/s eta 0:01:51
   ---------------------------------------- 0.1/38.8 MB 350.1 kB/s eta 0:01:51
   ---------------------------------------- 0.1/38.8 MB 350.1 kB/s eta 0:01:51
   ---------------------------------------- 0.2/38.8 MB 492.1 kB/s eta 0:01:19
   ---------------------------------------- 0.2/38.8 MB 492.1 kB/s eta 0:01:19
   ---------------------------------------- 0.2/38.8 MB 492.1 kB/s eta 0:01:19
   ------------------------

In [3]:
!python.exe -m pip install --upgrade pip

                                              0.0/1.8 MB ? eta -:--:--
                                              0.0/1.8 MB ? eta -:--:--
                                              0.0/1.8 MB ? eta -:--:--
                                              0.0/1.8 MB ? eta -:--:--
                                              0.0/1.8 MB ? eta -:--:--
                                              0.0/1.8 MB 163.4 kB/s eta 0:00:11
                                              0.0/1.8 MB 163.4 kB/s eta 0:00:11
                                              0.0/1.8 MB 163.4 kB/s eta 0:00:11
                                              0.0/1.8 MB 163.4 kB/s eta 0:00:11
     --                                       0.1/1.8 MB 288.1 kB/s eta 0:00:06
     --                                       0.1/1.8 MB 288.1 kB/s eta 0:00:06
     --                                       0.1/1.8 MB 288.1 kB/s eta 0:00:06
     --                                       0.1/1.8 MB 288.1 kB/s eta 0:00:06
     

In [7]:
!pip install numpy

In [11]:
import cv2
import numpy as np
from tkinter import *
from PIL import Image, ImageTk
from tkinter import ttk

# Load an image
image_path = 'girlBodBgRemoved.png'  # Replace with your image path
original_image = cv2.imread(image_path)
image = original_image.copy()

# Function to resize the image
def resize_body_parts(scale_leg, scale_hand, scale_neck, scale_shoulder):
    global image
    image = original_image.copy()
    height, width = image.shape[:2]

    # Resizing legs (bottom part of the image)
    leg_height = int(height * 0.25)
    leg_part = image[height - leg_height:, :]
    leg_part_resized = cv2.resize(leg_part, (width, int(leg_height * scale_leg)))
    new_leg_height = leg_part_resized.shape[0]

    # Resizing hands (middle part of the image)
    hand_height = int(height * 0.3)
    hand_part = image[hand_height:int(height * 0.75), :]
    hand_part_resized = cv2.resize(hand_part, (width, int(hand_height * scale_hand)))
    new_hand_height = hand_part_resized.shape[0]

    # Resizing neck (top middle part of the image)
    neck_height = int(height * 0.1)
    neck_part = image[int(height * 0.2):int(height * 0.3), :]
    neck_part_resized = cv2.resize(neck_part, (width, int(neck_height * scale_neck)))
    new_neck_height = neck_part_resized.shape[0]

    # Resizing shoulders (top part of the image)
    shoulder_height = int(height * 0.2)
    shoulder_part = image[0:int(height * 0.2), :]
    shoulder_part_resized = cv2.resize(shoulder_part, (width, int(shoulder_height * scale_shoulder)))
    new_shoulder_height = shoulder_part_resized.shape[0]

    # Create a new blank image
    new_image_height = new_shoulder_height + new_neck_height + new_hand_height + new_leg_height + int(height * 0.1)
    new_image = np.zeros((new_image_height, width, 3), dtype=np.uint8)

    # Place the resized parts in the new image
    current_y = 0

    # Place shoulders
    new_image[current_y:current_y + new_shoulder_height, :] = shoulder_part_resized
    current_y += new_shoulder_height

    # Place neck
    new_image[current_y:current_y + new_neck_height, :] = neck_part_resized
    current_y += new_neck_height

    # Place hands
    new_image[current_y:current_y + new_hand_height, :] = hand_part_resized
    current_y += new_hand_height

    # Place legs
    new_image[current_y:current_y + new_leg_height, :] = leg_part_resized

    image = new_image
    update_image_display()

# Function to update the image display
def update_image_display():
    global image, photo
    b, g, r = cv2.split(image)
    img = cv2.merge((r, g, b))
    img = Image.fromarray(img)
    img = ImageTk.PhotoImage(image=img)
    image_canvas.create_image(0, 0, anchor=NW, image=img)
    image_canvas.image = img

# Create the main window
root = Tk()
root.title("Body Part Resizer")

# Create a frame for the canvas and scrollbar
frame = Frame(root)
frame.pack(fill=BOTH, expand=1)

# Add a canvas widget
canvas = Canvas(frame)
canvas.pack(side=LEFT, fill=BOTH, expand=1)

# Add a scrollbar to the canvas
scrollbar = ttk.Scrollbar(frame, orient=VERTICAL, command=canvas.yview)
scrollbar.pack(side=RIGHT, fill=Y)

# Configure the canvas
canvas.configure(yscrollcommand=scrollbar.set)
canvas.bind('<Configure>', lambda e: canvas.configure(scrollregion=canvas.bbox("all")))

# Create another frame inside the canvas
frame_canvas = Frame(canvas)

# Add the new frame to a window in the canvas
canvas.create_window((0,0), window=frame_canvas, anchor="nw")

# Create sliders for each body part
scale_leg = Scale(frame_canvas, from_=0.5, to=1.5, resolution=0.1, orient=HORIZONTAL, label="Legs", command=lambda val: resize_body_parts(scale_leg.get(), scale_hand.get(), scale_neck.get(), scale_shoulder.get()))
scale_leg.set(1)
scale_leg.pack()

scale_hand = Scale(frame_canvas, from_=0.5, to=1.5, resolution=0.1, orient=HORIZONTAL, label="Torso", command=lambda val: resize_body_parts(scale_leg.get(), scale_hand.get(), scale_neck.get(), scale_shoulder.get()))
scale_hand.set(1)
scale_hand.pack()

scale_neck = Scale(frame_canvas, from_=0.5, to=1.5, resolution=0.1, orient=HORIZONTAL, label="Upper Torso", command=lambda val: resize_body_parts(scale_leg.get(), scale_hand.get(), scale_neck.get(), scale_shoulder.get()))
scale_neck.set(1)
scale_neck.pack()

scale_shoulder = Scale(frame_canvas, from_=0.5, to=1.5, resolution=0.1, orient=HORIZONTAL, label="Head", command=lambda val: resize_body_parts(scale_leg.get(), scale_hand.get(), scale_neck.get(), scale_shoulder.get()))
scale_shoulder.set(1)
scale_shoulder.pack()

# Add a canvas to display the image
image_canvas = Canvas(frame_canvas, width=original_image.shape[1], height=original_image.shape[0])
image_canvas.pack()

# Initialize image display
update_image_display()

# Start the GUI loop
root.mainloop()

In [13]:
import cv2
import numpy as np
from tkinter import *
from PIL import Image, ImageTk
from tkinter import ttk

# Load an image
image_path = 'girlBodBgRemoved.png'  # Replace with your image path
original_image = cv2.imread(image_path)
image = original_image.copy()

# Function to resize the image
def resize_body_parts(scale_leg, scale_hand, scale_neck, scale_shoulder):
    global image
    image = original_image.copy()
    height, width = image.shape[:2]

    # Resizing legs (bottom part of the image)
    leg_height = int(height * 0.25)
    leg_part = image[height - leg_height:, :]
    leg_part_resized = cv2.resize(leg_part, (width, int(leg_height * scale_leg)))
    new_leg_height = leg_part_resized.shape[0]

    # Resizing hands (middle part of the image)
    hand_height = int(height * 0.3)
    hand_part = image[hand_height:int(height * 0.75), :]
    hand_part_resized = cv2.resize(hand_part, (width, int(hand_height * scale_hand)))
    new_hand_height = hand_part_resized.shape[0]

    # Resizing neck (top middle part of the image)
    neck_height = int(height * 0.1)
    neck_part = image[int(height * 0.2):int(height * 0.3), :]
    neck_part_resized = cv2.resize(neck_part, (width, int(neck_height * scale_neck)))
    new_neck_height = neck_part_resized.shape[0]

    # Resizing shoulders (top part of the image)
    shoulder_height = int(height * 0.2)
    shoulder_part = image[0:int(height * 0.2), :]
    shoulder_part_resized = cv2.resize(shoulder_part, (width, int(shoulder_height * scale_shoulder)))
    new_shoulder_height = shoulder_part_resized.shape[0]

    # Create a new blank image
    new_image_height = new_shoulder_height + new_neck_height + new_hand_height + new_leg_height + int(height * 0.1)
    new_image = np.zeros((new_image_height, width, 3), dtype=np.uint8)

    # Place the resized parts in the new image
    current_y = 0

    # Place shoulders
    new_image[current_y:current_y + new_shoulder_height, :] = shoulder_part_resized
    current_y += new_shoulder_height

    # Place neck
    new_image[current_y:current_y + new_neck_height, :] = neck_part_resized
    current_y += new_neck_height

    # Place hands
    new_image[current_y:current_y + new_hand_height, :] = hand_part_resized
    current_y += new_hand_height

    # Place legs
    new_image[current_y:current_y + new_leg_height, :] = leg_part_resized

    image = new_image
    update_image_display()

# Function to update the image display
def update_image_display():
    global image, photo
    b, g, r = cv2.split(image)
    img = cv2.merge((r, g, b))
    img = Image.fromarray(img)
    img = ImageTk.PhotoImage(image=img)
    image_canvas.create_image(0, 0, anchor=NW, image=img)
    image_canvas.image = img

# Create the main window
root = Tk()
root.title("Body Part Resizer")

# Create a frame for the image canvas
image_frame = Frame(root)
image_frame.pack()

# Add a canvas to display the image
image_canvas = Canvas(image_frame, width=original_image.shape[1], height=original_image.shape[0])
image_canvas.pack()

# Initialize image display
update_image_display()

# Create a frame for the sliders
slider_frame = Frame(root)
slider_frame.pack()

# Create sliders for each body part
scale_leg = Scale(slider_frame, from_=0.5, to=1.5, resolution=0.1, orient=HORIZONTAL, label="Legs", command=lambda val: resize_body_parts(scale_leg.get(), scale_hand.get(), scale_neck.get(), scale_shoulder.get()))
scale_leg.set(1)
scale_leg.pack(side=LEFT, padx=10, pady=10)

scale_hand = Scale(slider_frame, from_=0.5, to=1.5, resolution=0.1, orient=HORIZONTAL, label="Torso", command=lambda val: resize_body_parts(scale_leg.get(), scale_hand.get(), scale_neck.get(), scale_shoulder.get()))
scale_hand.set(1)
scale_hand.pack(side=LEFT, padx=10, pady=10)

scale_neck = Scale(slider_frame, from_=0.5, to=1.5, resolution=0.1, orient=HORIZONTAL, label="Upper Torso", command=lambda val: resize_body_parts(scale_leg.get(), scale_hand.get(), scale_neck.get(), scale_shoulder.get()))
scale_neck.set(1)
scale_neck.pack(side=LEFT, padx=10, pady=10)

scale_shoulder = Scale(slider_frame, from_=0.5, to=1.5, resolution=0.1, orient=HORIZONTAL, label="Head", command=lambda val: resize_body_parts(scale_leg.get(), scale_hand.get(), scale_neck.get(), scale_shoulder.get()))
scale_shoulder.set(1)
scale_shoulder.pack(side=LEFT, padx=10, pady=10)

# Start the GUI loop
root.mainloop()